In [ ]:
import cv2
import numpy as np
import os
from imutils import paths
from nd2reader import ND2Reader

# K-means

First, we define the path where the images are located and where the segmentation of those images will be saved. In addition, we have to specify the extension of the images.  

In [ ]:
path = 'NuevasHector/images/Experiment0/'
outputPath = 'NuevasHector/kmeans/'
os.makedirs(outputPath,exist_ok=True)
ext = '.tif'

The following code perform the segmentation process using the K-means algorithm. 

In [ ]:
numberOfClusters = 2
stopCriteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.1)

for imPath in paths.list_files(path,validExts=ext):
    name = imPath[imPath.rfind('/')+1:imPath.rfind('.')]
    if(ext=='.nd2'):
        originalImage = ND2Reader(imPath)[0]
    else:
        originalImage = cv2.imread(imPath)

    originalImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2RGB)
    reshapedImage = np.float32(originalImage.reshape(-1, 3))
    
    ret, labels, clusters = cv2.kmeans(reshapedImage, numberOfClusters, None, stopCriteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    clusters = np.uint8(clusters)
    intermediateImage = clusters[labels.flatten()]
    clusteredImage = intermediateImage.reshape((originalImage.shape))
    cv2.imwrite(outputPath+name+'.jpg', clusteredImage)

# Canny segmentation

The structure is the same as the K-means method. 

In [ ]:
path = 'NuevasHector/images/Experiment0/'
outputPath = 'NuevasHector/canny/'
os.makedirs(outputPath,exist_ok=True)
ext = '.tif'

In [ ]:
for imPath in paths.list_images(path):
    name = imPath[imPath.rfind('/')+1:imPath.rfind('.')]
    if(ext=='.nd2'):
        originalImage = ND2Reader(imPath)[0]
    else:
        originalImage = cv2.imread(imPath)
    cannyImage = cv2.Canny(originalImage,0,50)
    cannyImage = cv2.dilate(cannyImage,(11,11))
    cnts,_ = cv2.findContours(cannyImage.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    im2 = np.zeros(cannyImage.shape[0:2], dtype = "uint8")
    cv2.fillPoly(im2,cnts,255)
    cv2.imwrite(outputPath+name+'.jpg', im2)